Intialization

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

Loading Table

In [0]:
df_silver = spark.table("workspace.bronze.erp_cust_az12")

Exploring Data

In [0]:
df_silver.summary("count").show()
df_silver.summary().show()

In [0]:
df_silver.select("GEN").distinct().display()

Trimming Data

In [0]:
for field in df_silver.schema.fields:
    if isinstance(field.dataType, StringType):
        df_silver = df_silver.withColumn(field.name, trim(col(field.name)))

Cleaning Data

In [0]:
#modify cid column so we can join it to another table
df_silver = df_silver.withColumn(
    "cid",
    F.when(col("cid").startswith("NAS"),
           F.substring(col("cid"), 4, F.length(col("cid"))))
     .otherwise(col("cid"))
)


In [0]:

df_silver = df_silver.withColumn(
    "bdate",
    F.when(col("bdate") > F.current_date(), None)
     .otherwise(col("bdate"))
)

In [0]:

df_silver = df_silver.withColumn(
    "gen",
    F.when(F.upper(col("gen")).isin("F", "FEMALE"), "Female")
     .when(F.upper(col("gen")).isin("M", "MALE"), "Male")
     .otherwise("n/a")
)

Renaming Columns

In [0]:
RENAME_MAP = {
    "cid": "customer_number",
    "bdate": "birth_date",
    "gen": "gender"
}
for old_name, new_name in RENAME_MAP.items():
    df_silver = df_silver.withColumnRenamed(old_name, new_name)

In [0]:
df_silver.limit(10).display()

In [0]:
df_silver.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customers")